In [144]:
import pandas as pd
import numpy as np

In [ ]:
def generate_sequential_id(df, columns): # генерирует последовательный ключ
    unique_values = df[columns].astype(str).apply(lambda x: '_'.join(x), axis=1)
    
    unique_mapping = {val: idx for idx, val in enumerate(sorted(unique_values.unique()), start=1)}
    
    return unique_values.map(unique_mapping)

In [ ]:
file_path = "data/customer_and_transaction.xlsx" # чтение листов xlsx
customers_df = pd.read_excel(file_path, sheet_name="customer")
transactions_df = pd.read_excel(file_path, sheet_name="transaction")

In [ ]:
# выполняется генерация уникальных ключей без потери связности
columns = ['brand', 'product_line', 'product_class', 'product_size', 'standard_cost']
transactions_df['new_product_id'] = generate_sequential_id(transactions_df, columns)

# обновляем product_id
transactions_df.drop(columns=['product_id'], inplace=True)
transactions_df.rename(columns={'new_product_id': 'product_id'}, inplace=True)


In [ ]:
# анализ products_df
products_df = transactions_df[
    [
        "product_id",
        "brand",
        "product_line",
        "product_class",
        "product_size",
        "standard_cost",
    ]
]
print(f"Дубликаты строк таблицы products_df = {products_df.duplicated().sum()}")

products_df = products_df[
    ~products_df.duplicated(subset=["product_id"])
]
products_df.reset_index(drop=True, inplace=True)
print(f"Дубликаты строк таблицы products_df после очистки = {products_df.duplicated().sum()}")
products_df

Дубликаты строк таблицы products_df = 19896
Дубликаты строк таблицы products_df после очистки = 0


,product_id,brand,product_line,product_class,product_size,standard_cost
0,65,Solex,Standard,medium,medium,53.620000
1,82,Trek Bicycles,Standard,medium,large,388.920000
2,41,OHM Cycles,Standard,low,medium,248.820000
3,29,Norco Bicycles,Standard,medium,medium,381.100000
4,10,Giant Bicycles,Standard,medium,large,709.480000
...,...,...,...,...,...,...
99,22,Norco Bicycles,Road,medium,medium,407.540000
100,48,OHM Cycles,Standard,medium,medium,770.890000
101,51,Solex,Road,medium,medium,312.735016
102,27,Norco Bicycles,Standard,medium,medium,270.299988


In [125]:
transactions_df['product_id'].value_counts().describe()

count    104.000000
mean     192.307692
std       51.602679
min        1.000000
25%      183.000000
50%      191.500000
75%      204.750000
max      465.000000
Name: count, dtype: float64

In [126]:
products_df.sort_values(by='product_id', inplace=True)

In [127]:
products_df

,product_id,brand,product_line,product_class,product_size,standard_cost
93,1,Giant Bicycles,Road,low,small,525.33
50,2,Giant Bicycles,Road,medium,medium,594.68
5,3,Giant Bicycles,Road,medium,medium,829.65
43,4,Giant Bicycles,Standard,high,medium,677.48
48,5,Giant Bicycles,Standard,high,medium,707.40
...,...,...,...,...,...,...
46,100,WeareA2B,Standard,medium,small,131.92
76,101,WeareA2B,Touring,medium,large,260.14
17,102,WeareA2B,Touring,medium,large,57.74
84,103,WeareA2B,Touring,medium,medium,363.25


In [128]:
products_df.to_csv("data/product.csv", index=False, encoding='utf-8')
print(f"Создан файл: product.csv")

In [129]:
transactions_df.drop(["brand", "product_line", "product_class", "product_size", "standard_cost"], axis=1, inplace=True)

In [ ]:
# анализ transactions_df
transactions_df

,transaction_id,customer_id,transaction_date,online_order,order_status,list_price,product_id
0,1,2950,2017-02-25,False,Approved,71.49,65
1,2,3120,2017-05-21,True,Approved,2091.47,82
2,3,402,2017-10-16,False,Approved,1793.43,41
3,4,3135,2017-08-31,False,Approved,1198.46,29
4,5,787,2017-10-01,True,Approved,1765.30,10
...,...,...,...,...,...,...,...
19995,19996,1018,2017-06-24,True,Approved,2005.66,38
19996,19997,127,2017-11-09,True,Approved,416.98,52
19997,19998,2284,2017-04-14,True,Approved,1636.90,47
19998,19999,2764,2017-07-03,False,Approved,227.88,39


In [131]:
transactions_df.dtypes

transaction_id               int64
customer_id                  int64
transaction_date    datetime64[ns]
online_order                object
order_status                object
list_price                 float64
product_id                   int64
dtype: object

In [132]:
transactions_df["online_order"] = transactions_df["online_order"].astype("bool")

In [133]:
transactions_df.dtypes

transaction_id               int64
customer_id                  int64
transaction_date    datetime64[ns]
online_order                  bool
order_status                object
list_price                 float64
product_id                   int64
dtype: object

In [ ]:
transactions_df = transactions_df[transactions_df["customer_id"] <= 4000]

In [134]:
transactions_df.to_csv("data/transaction.csv", index=False, encoding='utf-8')
print(f"Создан файл: transaction.csv")

In [ ]:
# анализ customers_df: address_df
customers_df

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09 00:00:00,VP Product Management,Health,Mass Customer,N,No,57042 Village Green Point,4511,QLD,Australia,6
3996,3997,Blanch,Nisuis,Female,2001-07-13 00:00:00,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,3998,Sarene,Woolley,U,NaN,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
3998,3999,Patrizius,NaN,Male,1973-10-24 00:00:00,NaN,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7


In [ ]:
columns = ['customer_id', 'address', 'postcode', 'state', 'country']
customers_df['address_id'] = generate_sequential_id(customers_df, columns) # генерация ключа

In [ ]:
# анализ address_df
address_df = customers_df[
    [
        "address_id",
        "customer_id",
        "address",
        "postcode",
        "state",
        "country",
    ]
]
print(f"Дубликаты строк таблицы address_df = {address_df.duplicated().sum()}")

address_df = address_df[
    ~address_df.duplicated(subset=["address_id"])
]
print(f"Дубликаты строк таблицы address_df после очистки = {address_df.duplicated().sum()}")
address_df.sort_values(by='address_id', inplace=True)
address_df

Дубликаты строк таблицы address_df = 0
Дубликаты строк таблицы address_df после очистки = 0


,address_id,customer_id,address,postcode,state,country
999,1,1000,61 Lukken Plaza,3182,VIC,Australia
1000,2,1001,67 Holmberg Way,2165,NSW,Australia
1001,3,1002,3383 Fair Oaks Circle,3046,VIC,Australia
1002,4,1003,2 Vermont Avenue,2066,NSW,Australia
1003,5,1004,486 Atwood Pass,2065,NSW,Australia
...,...,...,...,...,...,...
996,3996,997,27 Sunfield Center,4869,QLD,Australia
997,3997,998,656 Bartelt Parkway,3127,VIC,Australia
998,3998,999,610 Lakewood Gardens Point,4078,QLD,Australia
98,3999,99,68 Little Fleur Crossing,2046,New South Wales,Australia


In [138]:
address_df.to_csv("data/address.csv", index=False, encoding='utf-8')
print(f"Создан файл: address.csv")

In [139]:
customers_df.drop(["address_id", "address", "postcode", "state", "country"], axis=1, inplace=True)

In [ ]:
# анализ customers_df: financial_situation_df
customers_df

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9
...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09 00:00:00,VP Product Management,Health,Mass Customer,N,No,6
3996,3997,Blanch,Nisuis,Female,2001-07-13 00:00:00,Statistician II,Manufacturing,High Net Worth,N,Yes,10
3997,3998,Sarene,Woolley,U,NaN,Assistant Manager,IT,High Net Worth,N,No,7
3998,3999,Patrizius,NaN,Male,1973-10-24 00:00:00,NaN,Manufacturing,Affluent Customer,N,Yes,7


In [141]:
customers_df["owns_car"].value_counts()

owns_car
Yes    2024
No     1976
Name: count, dtype: int64

In [142]:
customers_df["deceased_indicator"].value_counts()

deceased_indicator
N    3998
Y       2
Name: count, dtype: int64

In [ ]:
# маппинг (bool)
customers_df['owns_car'] = np.where(customers_df['owns_car'] == 'Yes', True, False)
customers_df['deceased_indicator'] = np.where(customers_df['deceased_indicator'] == 'Y', True, False)

In [146]:
customers_df

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,False,True,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,False,True,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,False,True,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,False,False,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,False,True,9
...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09 00:00:00,VP Product Management,Health,Mass Customer,False,False,6
3996,3997,Blanch,Nisuis,Female,2001-07-13 00:00:00,Statistician II,Manufacturing,High Net Worth,False,True,10
3997,3998,Sarene,Woolley,U,NaN,Assistant Manager,IT,High Net Worth,False,False,7
3998,3999,Patrizius,NaN,Male,1973-10-24 00:00:00,NaN,Manufacturing,Affluent Customer,False,True,7


In [147]:
financial_situation_df = customers_df[
    [
        "customer_id",
        "job_title",
        "job_industry_category",
        "wealth_segment",
        "deceased_indicator",
        "owns_car",
        "property_valuation"
    ]
]
print(f"Дубликаты строк таблицы financial_situation_df = {financial_situation_df.duplicated().sum()}")

financial_situation_df = financial_situation_df[
    ~financial_situation_df.duplicated(subset=["customer_id"])
]
print(f"Дубликаты строк таблицы financial_situation_df после очистки = {financial_situation_df.duplicated().sum()}")
financial_situation_df.sort_values(by='customer_id', inplace=True)
financial_situation_df

Дубликаты строк таблицы financial_situation_df = 0
Дубликаты строк таблицы financial_situation_df после очистки = 0


,customer_id,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,property_valuation
0,1,Executive Secretary,Health,Mass Customer,False,True,10
1,2,Administrative Officer,Financial Services,Mass Customer,False,True,10
2,3,Recruiting Manager,Property,Mass Customer,False,True,9
3,4,NaN,IT,Mass Customer,False,False,4
4,5,Senior Editor,NaN,Affluent Customer,False,True,9
...,...,...,...,...,...,...,...
3995,3996,VP Product Management,Health,Mass Customer,False,False,6
3996,3997,Statistician II,Manufacturing,High Net Worth,False,True,10
3997,3998,Assistant Manager,IT,High Net Worth,False,False,7
3998,3999,NaN,Manufacturing,Affluent Customer,False,True,7


In [149]:
financial_situation_df.dtypes

customer_id               int64
job_title                object
job_industry_category    object
wealth_segment           object
deceased_indicator         bool
owns_car                   bool
property_valuation        int64
dtype: object

In [148]:
financial_situation_df.to_csv("data/financial_situation.csv", index=False, encoding='utf-8')
print(f"Создан файл: financial_situation.csv")

In [150]:
customers_df.drop(["job_title", "job_industry_category", "wealth_segment", "deceased_indicator", "owns_car", "property_valuation"], axis=1, inplace=True)

In [ ]:
# анализ customers_df
customers_df

,customer_id,first_name,last_name,gender,DOB
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00
1,2,Eli,Bockman,Male,1980-12-16 00:00:00
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00
3,4,Talbot,NaN,Male,1961-10-03 00:00:00
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00
...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09 00:00:00
3996,3997,Blanch,Nisuis,Female,2001-07-13 00:00:00
3997,3998,Sarene,Woolley,U,NaN
3998,3999,Patrizius,NaN,Male,1973-10-24 00:00:00


In [153]:
customers_df.to_csv("data/customer.csv", index=False, encoding='utf-8')
print(f"Создан файл: customer.csv")